# GPT-5 Basic Test Notebook (Azure OpenAI)

This notebook provides examples of the Chat Completions API and the Responses API to quickly validate the basic functionality of GPT-5 in Azure OpenAI. It uses the Azure OpenAI endpoint, key, and deployment name configured in the environment variables.

## Requruirements

- Python 3.10+
- `pip install -r GPT-5/requirements.txt`
- `GPT-5/.env` 
  - AZURE_OPENAI_ENDPOINT
  - AZURE_OPENAI_API_KEY
  - AZURE_OPENAI_API_VERSION


## Chat Completions API Test

In [52]:
import os
import json
import openai
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, EnvironmentCredential
from azure.keyvault.secrets import SecretClient
from io import BytesIO
import gradio as gr
import time

import json
load_dotenv(override=True)

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY", "") if len(os.getenv("AZURE_OPENAI_API_KEY", "")) > 0 else None
azure_openai_gpt5_deployment_name = "gpt-5"
azure_openai_gpt41_deployment_name = "gpt-4.1"
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "") if len(os.getenv("AZURE_OPENAI_API_VERSION", "")) > 0 else None


try:
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_key=azure_openai_key,
        api_version=aoai_api_version
    )
except (ValueError, TypeError) as e:
    print(e)


# Print environment variables (excluding sensitive info)
print("=== Azure OpenAI 환경 설정 ===")
print(f"Endpoint: {azure_openai_endpoint}")
print(f"API Version: {aoai_api_version}")
print(f"Deployment Name: {azure_openai_gpt5_deployment_name}")
print(f"Embedding Deployment: {azure_openai_embedding_deployment}")
print(f"API Key set: {azure_openai_key is not None}")
print(f"Client initialized: {client is not None}")

=== Azure OpenAI 환경 설정 ===
Endpoint: https://hyo-ai-foundry-pjt1-resource.openai.azure.com/
API Version: 2025-03-01-preview
Deployment Name: gpt-5
Embedding Deployment: text-embedding-ada-002
API Key set: True
Client initialized: True


## Function Calling Test

In [ ]:
# 1) 기본 채팅
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Microsoft에 대한 가장 최신 정보를 알려줘."}
]

response = client.chat.completions.create(
    model=azure_openai_gpt5_deployment_name,
    messages=messages
)

print("=== 기본 채팅 예제 ===")
print(response.choices[0].message.content)
print()

# 2) 함수 호출 (간단한 버전)
def get_weather(city: str):
    """Mock weather function"""
    weather_data = {
        "Seoul": "맑음, 22°C",
        "Busan": "흐림, 19°C"
    }
    return weather_data.get(city, f"{city}: 날씨 정보 없음")

# 함수 스키마 정의
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get weather for a city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name"}
            },
            "required": ["city"]
        }
    }
}]

# 함수 호출이 포함된 대화
messages = [
    {"role": "user", "content": "서울 날씨 알려줘"}
]

print("=== 함수 호출 예제 ===")
response = client.chat.completions.create(
    model=azure_openai_gpt5_deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

# 함수 호출 확인 및 실행 (실제 코드는 while 루프 등으로 실행하는게 일반적)
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    print(f"함수 호출: {function_name}({arguments})")
    
    # 실제 함수 실행
    if function_name == "get_weather":
        result = get_weather(arguments["city"])
        print(f"함수 결과: {result}")
        
        # 함수 결과와 함께 최종 응답 생성
        messages.append(response.choices[0].message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })
        
        final_response = client.chat.completions.create(
            model=azure_openai_gpt5_deployment_name,
            messages=messages
        )
        
        print(f"최종 응답: {final_response.choices[0].message.content}")
else:
    print(f"직접 응답: {response.choices[0].message.content}")

print()


=== 기본 채팅 예제 ===
다음 내용은 제가 확인할 수 있는 최신 공개 정보(지식 기준: 2024년 10월)까지를 바탕으로 한 마이크로소프트(Microsoft) 동향 요약입니다. 2025년에 새로 발표된 사항은 포함되지 않았을 수 있으니, 정말 “최신” 소식을 원하시면 공식 뉴스룸이나 최근 실적 발표 자료를 함께 확인하시길 권합니다.

핵심 요약
- 전사 전략: “클라우드 + AI”에 집중. Copilot을 중심으로 제품 전반에 생성형 AI를 통합하고, 이를 뒷받침하기 위해 Azure 인프라(데이터센터·전력·네트워킹·반도체 파트너십)에 대규모 투자를 지속.
- 실적/투자: 클라우드와 AI 수요 확대에 힘입어 사상 최대 실적을 경신(연속 분기 최고치 갱신). AI 워크로드 대응을 위한 설비투자(CapEx)가 크게 증가하는 기조가 이어짐.
- 규제/정책: Teams 번들 이슈에 대한 비판에 대응해 2024년에 전세계적으로 Teams 분리판매를 확대. 유럽 등에서 클라우드 라이선싱 관행 관련 조사와 논의가 진행 중.
- 파트너십/거버넌스: OpenAI와의 전략적 파트너십 유지. 2024년 중반에는 각국 규제 당국의 우려를 고려해 OpenAI 이사회 옵저버 자리에서 물러났으나, 기술·사업 협력은 지속.
- 보안: 국가 연계 위협 사건 공개 이후 “Secure Future Initiative”를 통해 기본 보안 강화, 엔지니어링 프로세스 개선, 제품 기본 설정의 보안성 제고 등 전사적 보안 체질 개선을 진행.

AI와 Copilot
- Copilot 확장: Bing Chat/Office 365의 AI 기능을 “Copilot” 브랜드로 통합. Copilot Pro(개인 유료), Copilot for Microsoft 365(기업용) 보급이 확대되었고, 2024년에는 좌석 수 제한 완화 등 도입 장벽이 낮아짐.
- 팀 협업: 회의·프로젝트 관리·문서 협업에서 AI가 팀 단위로 보조하는 ‘Team Copilot’이 2024년에 공개/예고되며 기능 영역 확대.
- 개발자 

## Structured Output Test with Json Schema

In [33]:
# 5) 구조화된 출력 (Chat Completions API - JSON Schema)
json_schema = {
    "name": "product_summary",
    "schema": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "category": {"type": "string"},
            "price_krw": {"type": "number"},
            "pros": {"type": "array", "items": {"type": "string"}},
            "cons": {"type": "array", "items": {"type": "string"}},
        },
        "required": ["title", "category", "price_krw", "pros", "cons"], 
        "additionalProperties": False,
    },
    "strict": True,
}

messages = [
    {"role": "user", "content": "게이밍 노트북 추천 모델 하나를 임의로 정하고 요약 JSON을 생성해줘. 가격은 원화로 대략적 수치를 넣어."}
]

response = client.chat.completions.create(
    model=azure_openai_gpt5_deployment_name,
    messages=messages,
    response_format={"type": "json_schema", "json_schema": json_schema},
)

print("=== Structured Output (JSON Schema) ===")
print(response.choices[0].message.content)

=== Structured Output (JSON Schema) ===
{"title":"ASUS ROG Zephyrus G14 (2024, RTX 4070)","category":"게이밍 노트북","price_krw":2900000,"pros":["3K 120Hz OLED 디스플레이로 선명한 화질과 빠른 응답","RTX 4070 + 최신 Ryzen 9 조합으로 강력한 게임/크리에이티브 성능","1.6kg대의 경량·슬림 설계로 휴대성 우수","우수한 스피커와 키보드/트랙패드 품질","성능·소음 프로필 전환 등 유연한 팬/전력 관리"],"cons":["가격대가 높음","고부하 시 발열과 팬 소음 증가","메모리 업그레이드 제약(일부 용량 온보드)","포트 구성 제한적(유선 LAN 미탑재 등)","OLED 특성상 번인 우려 및 SDR 밝기 제한"]}


## Responses API (stateful chat for 30days)

In [34]:
response = client.responses.create(   
  model=azure_openai_gpt5_deployment_name, 
  input="This is a test.",
)

print(response.model_dump_json(indent=2)) 

{
  "id": "resp_68baa470744c819cbdf882d929a1b21f0db26a9495fbecdb",
  "created_at": 1757062256.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5",
  "object": "response",
  "output": [
    {
      "id": "rs_68baa470fee0819c9b6f84a4393abbe30db26a9495fbecdb",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68baa471b3c0819cbe84fd773233e64e0db26a9495fbecdb",
      "content": [
        {
          "annotations": [],
          "text": "Thanks for the test. How can I help you today?",
          "type": "output_text",
          "logprobs": null
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": false,
  "max_output_tokens": null,
  "max_tool_calls": 

## Responses API - Function Calling Test

In [35]:
response = client.responses.create(  
    model=azure_openai_gpt5_deployment_name, 
    tools=[  
        {  
            "type": "function",  
            "name": "get_weather",  
            "description": "Get the weather for a location",  
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "location": {"type": "string"},  
                },  
                "required": ["location"],  
            },  
        }  
    ],  
    input=[{"role": "user", "content": "What's the weather in San Francisco?"}],  
)  

print(response.model_dump_json(indent=2))  
  
# To provide output to tools, add a response for each tool call to an array passed  
# to the next response as `input`  
input = []  
for output in response.output:  
    if output.type == "function_call":  
        match output.name:  
            case "get_weather":  
                input.append(  
                    {  
                        "type": "function_call_output",  
                        "call_id": output.call_id,  
                        "output": '{"temperature": "70 degrees"}',  
                    }  
                )  
            case _:  
                raise ValueError(f"Unknown function call: {output.name}")  
  
second_response = client.responses.create(  
    model=azure_openai_gpt41_deployment_name,  
    previous_response_id=response.id,  
    input=input  
)  

print(second_response.model_dump_json(indent=2)) 

{
  "id": "resp_68baa47759e481a299f683a7eefa9e9303b6b99fee8d17a1",
  "created_at": 1757062263.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5",
  "object": "response",
  "output": [
    {
      "id": "rs_68baa47842c481a280bbd8b2ae5aa1be03b6b99fee8d17a1",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "arguments": "{\"location\":\"San Francisco\"}",
      "call_id": "call_35iBDrjMUCPw0suPvmGf9HrC",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_68baa479342c81a2ba49b544cab6995f03b6b99fee8d17a1",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string"
          }
        },


## Responses API - Structured Output Test with Json Schema

In [40]:
# 5) 구조화된 출력 (Responses API - JSON Schema)
# json_schema = {
#     "name": "product_summary",
#     "schema": {
#         "type": "object",
#         "properties": {
#             "title": {"type": "string"},
#             "category": {"type": "string"},
#             "price_krw": {"type": "number"},
#             "pros": {"type": "array", "items": {"type": "string"}},
#             "cons": {"type": "array", "items": {"type": "string"}},
#         },
#         "required": ["title", "category", "price_krw"],
#         "additionalProperties": False,
#     },
#     "strict": True,
# }

# inputs = [
#     {"role": "user", "content": "게이밍 노트북 추천 모델 하나를 임의로 정하고 요약 JSON을 생성해줘. 가격은 원화로 대략적 수치를 넣어."}
# ]

# response = client.responses.create(
#     model=azure_openai_gpt5_deployment_name,
#     input=inputs,
#     response_format={"type": "json_schema", "json_schema": json_schema},
# )

# print(response.model_dump_json(indent=2)) 

print("Responses API doen't support response_format.")


Responses API doen't support response_format.


## Responses API - Streaming output Test

In [53]:

try:
    stream = client.chat.completions.create(
        model=azure_openai_gpt5_deployment_name,
        messages=[{"role": "user", "content": "다섯줄 줄 농담 하나."}],
        stream=True,
    )
    
    for chunk in stream:
        if chunk.choices:
            delta = chunk.choices[0].delta.content
            if delta:
                print(delta, end="", flush=True)
                time.sleep(0.05)
    print()
    
except Exception as e:
    print(f"스트리밍 오류: {e}")

냉장고를 열었더니 물만 하나 있었다.
그래서 물어봤다.
"나 말고 뭐 먹을 거 없어?"
물이 대답했다.
"물론."


## Responses API - Vision Support Test

In [54]:
# 4) 멀티모달: 이미지 분석 (Responses API)
# Note: Requires the deployed model to support vision and your API version to allow image input.
image_url = os.getenv("TEST_IMAGE_URL", "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Fronalpstock_big.jpg/640px-Fronalpstock_big.jpg")

inputs = [
    {"role": "user", "content": [
        {"type": "input_text", "text": "이미지의 주요 특징을 한국어로 3줄 요약해줘."},
        {"type": "input_image", "image_url": image_url},
    ]}
]

response = client.responses.create(model=azure_openai_gpt5_deployment_name, input=inputs)

print(response.model_dump_json(indent=2)) 

{
  "id": "resp_68baa7048ac0819fbc24fd3dd1a2fce802610945b92f961a",
  "created_at": 1757062916.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5",
  "object": "response",
  "output": [
    {
      "id": "rs_68baa7070ddc819f9f16b5d03b9d8c4702610945b92f961a",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68baa70c7f64819fb75427bbd636d9b002610945b92f961a",
      "content": [
        {
          "annotations": [],
          "text": "- 초록 숲과 초원이 뒤덮인 산악 지형으로, 중앙의 절벽형 능선이 두드러집니다.\n- 산기슭에는 목초지와 작은 마을이 점점이 놓여 있으며 완만한 계곡이 펼쳐집니다.\n- 맑은 파란 하늘에 구름이 흩어져 있고, 멀리 더 높은 봉우리와 일부 설산이 보입니다.",
          "type": "output_text",
          "logprobs": null
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tool

## Responses API - Code Interpreter Test

In [55]:
instructions = "You are a personal math tutor. When asked a math question, write and run code using the python tool to answer the question."

response = client.responses.create(
    model=azure_openai_gpt5_deployment_name,
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }
    ],
    instructions=instructions,
    input="I need to solve the equation 3x + 11 = 14. Can you help me?",
)

print(response.model_dump_json(indent=2)) 

{
  "id": "resp_68baa72e3ae0819fa9555ff33ea9095807fb44d44c7f7b1f",
  "created_at": 1757062958.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a personal math tutor. When asked a math question, write and run code using the python tool to answer the question.",
  "metadata": {},
  "model": "gpt-5",
  "object": "response",
  "output": [
    {
      "id": "rs_68baa7307010819fad4220632f49d48307fb44d44c7f7b1f",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ci_68baa734b0f4819face6bf108689f32807fb44d44c7f7b1f",
      "code": "# Solve 3x + 11 = 14 for x\r\nx = (14 - 11) / 3\r\nx",
      "container_id": "cntr_68baa72f018c81908c456f9bb8c7a5620c3b65bc73b36f02",
      "outputs": null,
      "status": "completed",
      "type": "code_interpreter_call"
    },
    {
      "id": "msg_68baa736261c819fbae26878fdef040107fb44d44c7f7b1f",
      "content": [
        {
        

## Responses API - List previous chats

In [56]:
response = client.responses.input_items.list("resp_68baa72e3ae0819fa9555ff33ea9095807fb44d44c7f7b1f")

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_68baa72ea2a4819fabfe4bc7e6979c7907fb44d44c7f7b1f",
      "content": [
        {
          "text": "I need to solve the equation 3x + 11 = 14. Can you help me?",
          "type": "input_text"
        }
      ],
      "role": "user",
      "status": "completed",
      "type": "message"
    }
  ],
  "has_more": false,
  "object": "list",
  "first_id": "msg_68baa72ea2a4819fabfe4bc7e6979c7907fb44d44c7f7b1f",
  "last_id": "msg_68baa72ea2a4819fabfe4bc7e6979c7907fb44d44c7f7b1f"
}


## Responses API - Upload PDF and analyze

In [58]:
# Upload a file with a purpose of "assistants"
file = client.files.create(
  file=open("AI_Brief_Aug2025.pdf", "rb"), # This assumes a .pdf file in the same directory as the executing script
  purpose="assistants"
)

print(file.model_dump_json(indent=2))
file_id = file.id

{
  "id": "assistant-8StXmwbgba1Ke52mPt7AHs",
  "bytes": 1165436,
  "created_at": 1757063685,
  "filename": "AI_Brief_Aug2025.pdf",
  "object": "file",
  "purpose": "assistants",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}


In [61]:
response = client.responses.create(
    model=azure_openai_gpt5_deployment_name,
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id":"assistant-8StXmwbgba1Ke52mPt7AHs"
                },
                {
                    "type": "input_text",
                    "text": "Summarize this PDF in Korean in three sentences.",
                },
            ],
        },
    ]
)
print(response.model_dump_json(indent=2))

{
  "id": "resp_68baab59766881a2ac3a1c5dbdcd060c0a4857b25b5d767c",
  "created_at": 1757064029.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5",
  "object": "response",
  "output": [
    {
      "id": "rs_68baab6476ec81a2b4bc2e7a8cef48d30a4857b25b5d767c",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68baabb5917481a2abc7d0639f8226b50a4857b25b5d767c",
      "content": [
        {
          "annotations": [],
          "text": "- 정책·법제에선 대만 디지털발전부의 5대 정책도구(컴퓨팅 파워·데이터·인재·마케팅·자금) 발표, OECD의 생성 AI를 범용기술로 본 생산성·정책 권고, 미 상원의 주정부 AI 규제 10년 유예 조항 삭제, 트럼프 행정부의 AI 행동계획·3개 행정명령 공표, EU의 AI법 범용 AI 실천 강령·지침 공개가 핵심입니다. \n- 산업에선 미드저니 비디오 생성 ‘V1’ 출시, 바이두의 어니·딥시크 기반 AI 검색 대개편과 MCP 생태계 확장, 문샷AI의 에이전트형 오픈소스 ‘키미 K2’, xAI ‘그록 4’와 정부 시장 진출(국방부 계약), 퍼플렉시티의 에이전트 브라우저 ‘코멧’(한계 지적), 오픈AI의 복잡 업무를 수행하는 ‘챗GPT 에이전트’ 공개가 두드러졌습니다. \n- 기술·연구·인력 측면에선 딥마인드 

## Responses API - Background Task (async)

In [65]:
response = client.responses.create(
    model = azure_openai_gpt5_deployment_name,
    input = "Write me a bed time story (1min)",
    background = True
)

print(response.model_dump_json(indent=2))

{
  "id": "resp_68baaca73eac8191be4d9c743a09c8c50bfaaa0cc82df074",
  "created_at": 1757064360.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5",
  "object": "response",
  "output": [],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": true,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": {
    "effort": "medium",
    "generate_summary": null,
    "summary": null
  },
  "safety_identifier": null,
  "service_tier": "auto",
  "status": "queued",
  "text": {
    "format": {
      "type": "text"
    },
    "verbosity": null
  },
  "top_logprobs": null,
  "truncation": "disabled",
  "usage": null,
  "user": null,
  "content_filters": null,
  "store": true
}


In [66]:
from time import sleep

while response.status in {"queued", "in_progress"}:
    print(f"Current status: {response.status}")
    sleep(2)
    response = client.responses.retrieve(response.id)

print(f"Final status: {response.status}\nOutput:\n{response.output_text}")

Current status: queued


Current status: queued
Current status: queued
Current status: queued
Final status: completed
Output:
Tonight, the moon props a silver ladder by your window. You climb it with a little yawn, and at the top waits Luma, a pocket-sized keeper of light with a lantern no bigger than a teardrop. Together you drift in a thistledown boat across the quiet sky, where stars hum like sleepy bees and clouds breathe slow and soft.

Luma shows you jars of dreams, each glowing a different kind of gentle—vanilla, rain, warm bread, sunlight on closed eyes. You tip the dreams over rooftops, and the town below sighs as they fall like soft snow. An owl folds the night into its wings. A distant whale-shaped constellation hums a lullaby that twinkles through the air.

When the lantern grows warm, Luma smiles. Time to float home before dawn slips a pale feather into the east. You climb down the silver ladder. Your bed is a small boat now, your pillow a cloud, your blanket a tide.

Luma tucks one glimmer in you